In [1]:
import pandas as pd
import matplotlib.pylab as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
frame_test = pd.read_csv("hq_markup_train.csv")
frame_test.fillna(-2, inplace=True)
# frame_test["Граница постоянного давления_details"] = np.log10(frame_test["Граница постоянного давления_details"])
# frame_test["Граница непроницаемый разлом_details"] = np.log10(frame_test["Граница непроницаемый разлом_details"])
# frame_test.fillna(-0.5, inplace=True)
frame = pd.read_csv("markup_train.csv")
frame.fillna(-2, inplace=True)

def get_sample(index, frame):
    uuid = frame.iloc[index, 0]
    with open(f'data/{uuid}', 'r') as f:
        content = f.read()
    rows = [line.split('\t') for line in content.strip().split('\n')]
    data = pd.DataFrame(rows, columns=['time', 'delta_p', 'p_'], dtype=float)
    return data


def find_empty_indexes_2(data):
    """
    Поиск id файлов из строк датафрейма для их исключения на этапе удаления данных из датафрейма
    """
    B=[]
    for i in range(data.shape[0]):
        try:
            df=get_sample(i,data)
        except ValueError:
            B.append(data.iloc[i,0])
    return B


# class SiamDataset(Dataset):
#     def __init__(self, siam_dataset_describe:pd.DataFrame):
#         super().__init__()
#         self.siam_dataset_describe = siam_dataset_describe

#     def __len__(self):
#         return self.siam_dataset_describe.shape[0]

#     def __getitem__(self, idx):
#         x = get_sample(idx, self.siam_dataset_describe) #.to_numpy(dtype=np.float64)
#         # t = x["time"].to_numpy(dtype=np.float64)
#         x = x[["delta_p", "p_"]].to_numpy(dtype=np.float64)
        


#         # 7) Возвращаем (X, Y)                                                  # Давление(атм) ([:, 1]) Давление(атм) ([:,1]) Давление(атм) ([:,1]) Давление(атм) ([:,1]) Давление(атм) ([:,1]) В какой момент? [:, 0]   В какой момент? [:, 0]
#         return x, self.siam_dataset_describe.iloc[idx][['Влияние ствола скважины_details', 'Радиальный режим_details', 'Линейный режим_details', 'Билинейный режим_details', 'Сферический режим_details', 'Граница постоянного давления_details', 'Граница непроницаемый разлом_details']].to_numpy(dtype=np.float32)

B=find_empty_indexes_2(frame)#Поиск id с отсутствующими файлами для исключения
frame=frame[~frame['file_name'].isin(B)]#Фильтрация данных
frame = frame[~frame["file_name"].isin(frame_test["file_name"])]

In [11]:
frame.isna().any().any()

np.False_

In [2]:

class PositionalEncoding(nn.Module):
    """
    Классическое позиционное кодирование (NLP-стиль).
    """
    def __init__(self, d_model, max_len=4000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() 
                             * -(torch.log(torch.tensor(10000.0)) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe / (d_model ** 0.5)
        pe = pe.unsqueeze(0)    # [1, max_len, d_model]
        self.register_buffer('pe', pe) 

    def forward(self, x):
        """
        x shape: [B, T, d_model]
        Добавляем позиционное кодирование к x.
        """
        seq_len = x.size(1)
        x = x + self.pe[:, :seq_len, :]
        # print(f"Encoded x: {x}")
        return x

class TransformerFlowModel(nn.Module):
    """
    Пример трансформера для задач:
      - 8 бинарных признаков (классификация)
      - 7 регрессионных выходов
    С учётом attention mask (padding).
    """
    def __init__(self,
                 input_dim=3,     # (delta_p, p_, log_time) например
                 d_model=64,
                 nhead=4,
                 num_layers=2,
                 dim_feedforward=128,
                 dropout=0.1,
                 n_class=8,
                 n_reg=5):
        super().__init__()
        self.input_proj = nn.Linear(input_dim, d_model)
        # self.pos_encoder = PositionalEncoding(d_model)
        torch.nn.init.xavier_uniform_(self.input_proj.weight, gain=1.0)
        torch.nn.init.zeros_(self.input_proj.bias)

        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model,
                                                   nhead=nhead,
                                                   dim_feedforward=dim_feedforward,
                                                   dropout=dropout,
                                                   activation="relu",
                                                   batch_first=True)  
        self.transformer_encoder = nn.TransformerEncoder(
            encoder_layer, num_layers=num_layers)

        self.pool = nn.AdaptiveAvgPool1d(1)  # mean-pool по временной оси

        self.class_head = nn.Linear(d_model, n_class)
        self.reg_head   = nn.Linear(d_model, n_reg)

    def forward(self, x, src_key_padding_mask=None):
        """
        x: [B, T, input_dim]
        src_key_padding_mask: [B, T], True = игнорируем (паддинг)
        """
        # 1) Линейная проекция входа -> d_model
        x_proj = self.input_proj(x)  # [B, T, d_model]
        if torch.isnan(x_proj).any():
            print("NaN after input projection")
        # 2) Позиционное кодирование
        # x_encoded = self.pos_encoder(x_proj)  # [B, T, d_model]
        # if torch.isnan(x_encoded).any():
            # print("NaN after positional encoding")
        # 3) Пропускаем через энкодер
        #    Важно: указываем mask=... (или src_key_padding_mask=...).
        #    Маска должна быть типа bool, shape [B, T]
        x_trans = self.transformer_encoder(
            # x_encoded,
            x_proj,
            src_key_padding_mask=src_key_padding_mask
        )  # [B, T, d_model]
        if torch.isnan(x_trans).any():
            print("NaN after transformer")

        # 4) Pooling, [B, d_model]
        x_trans_perm = x_trans.permute(0, 2, 1)  # -> [B, d_model, T]
        pooled = self.pool(x_trans_perm).squeeze(-1)  # [B, d_model]
        if torch.isnan(pooled).any():
            print("NaN after pooling")

        # 5) Выход
        class_logits = self.class_head(pooled)  # [B, 8]
        reg_output   = self.reg_head(pooled)    # [B, 7]
        if torch.isnan(pooled).any():
            print("NaN after class_logits")
        return class_logits, reg_output


class ConvTransformer(nn.Module):
    def __init__(self, d_model=32, nhead=2, num_layers=1, dim_feedforward = 128, in_channels = 2):
        super().__init__()
        # 1) Conv1d: in_channels=2 (delta_p, p_), out_channels=d_model
        #    kernel_size=3 -> можно менять
        self.conv = nn.Conv1d(in_channels=in_channels, out_channels=d_model, kernel_size=31, padding=15, groups=in_channels)
        self.in_channels = in_channels
        # 2) TransformerEncoder (упрощённый)    
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            dropout=0.0,
            activation='relu',
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(
            encoder_layer,
            num_layers=num_layers
        )

        # 3) Heads для классификации (8) и регрессии (7)
        # self.class_head = nn.Linear(d_model, 8)
        self.reg_preload   = nn.Linear(d_model, 64)
        self.reg_activate = nn.LeakyReLU(0.3)
        self.reg_head   = nn.Linear(64, 1)

    def forward(self, x, src_key_padding_mask=None):
        """
        x: [B, T, 2]
        src_key_padding_mask: [B, T] (bool), True=игнорировать позицию
        """
        # -- (A) Свёртка --
        # Conv1d ожидает [B, C, T], значит permute:
        x = x.permute(0, 2, 1)  # -> [B, 2, T]

        # Прогон через conv:
        # Выход будет [B, d_model, T]
        x_conv = self.conv(x)  # [B, d_model, T]

        # -- (B) Для Transformer делаем [B, T, d_model]
        x_conv = x_conv.permute(0, 2, 1)  # [B, T, d_model]

        # -- (C) Прогон через TransformerEncoder, с учётом mask
        x_trans = self.transformer_encoder(
            x_conv,
            src_key_padding_mask=src_key_padding_mask
        )  # [B, T, d_model]

        # -- (D) Возьмём, например, средний вектор по времени
        x_pooled = x_trans.mean(dim=1)  # [B, d_model]

        # -- (E) Предсказываем 8 бинарных признаков + 7 регрессий
        # class_logits = self.class_head(x_pooled)  # [B, 8]
        reg_out      = self.reg_head(self.reg_activate(self.reg_preload(x_pooled)))    # [B, 7]
        return reg_out
        # return class_logits, reg_out



class SiamDataset(Dataset):
    def __init__(self, siam_dataset_describe:pd.DataFrame):
        super().__init__()
        self.siam_dataset_describe = siam_dataset_describe

    def __len__(self):
        return self.siam_dataset_describe.shape[0]

    def __getitem__(self, idx):
        x = get_sample(idx, self.siam_dataset_describe)
        # x = torch.from_numpy(get_sample(idx, self.siam_dataset_describe).to_numpy(dtype=np.float32)) #.to_numpy(dtype=np.float64)
        # t = x["time"].to_numpy(dtype=np.float64)
        def calculate_log_pressure(pressure):
            """Calculate log of pressure for log-log plots"""
            # Ensure all pressure values are positive
            min_positive = np.min(pressure[pressure > 0]) / 10 if any(pressure > 0) else 1e-10
            adjusted_pressure = np.maximum(pressure, min_positive)
            return np.log10(adjusted_pressure)
        
        x["time"] = np.log10(x["time"])
        x["delta_p"] = calculate_log_pressure(x["delta_p"])
        x["p_"] = calculate_log_pressure(x["p_"]) 
                         # "time",   "delta_p",
        x =  torch.from_numpy(x[["time", "delta_p", "p_"]].to_numpy(dtype=np.float32))
        
        # x =  torch.from_numpy(x[["p_"]].to_numpy(dtype=np.float32))
        if torch.isnan(x).any():
            # print(get_sample(idx, self.siam_dataset_describe))
            mask = ~torch.isnan(x).any(dim=1)
            x = x[~mask]

            
            # print(x)
            # raise Exception("ахтунг в данных")
        if x.shape[0] == 0:
            x = torch.tensor([0,0,0])


        # 7) Возвращаем (X, Y)                                                                                                                                              # не работает если ставить 1 число
        return x, torch.from_numpy(self.siam_dataset_describe.iloc[idx][self.siam_dataset_describe.columns[3:11]].to_numpy(dtype=np.float32)), torch.from_numpy(self.siam_dataset_describe.iloc[idx][self.siam_dataset_describe.columns[11:]].to_numpy(dtype=np.float32))



def collate_fn_with_padding(batch):
    """
    batch: список из (X, y_class, y_reg), где X shape [T_i, input_dim].
    Нужно вернуть:
      padded_X: [B, max_len, input_dim]
      src_key_padding_mask: [B, max_len] (bool)
      y_class: [B, 8]
      y_reg:   [B, 7]
    """
    # 1) Определяем batch_size
    batch_size = len(batch)
    # 2) Находим максимальную длину среди X
    lengths = [sample[0].shape[0] for sample in batch]  # T_i для каждого
    max_len = max(lengths)
    input_dim = batch[0][0].shape[1]

    # 3) Создаём тензоры под результирующие данные
    padded_X = torch.zeros((batch_size, max_len, input_dim), dtype=torch.float)
    # Маска: True = игнорируем => паддинг
    # Изначально False (значит реальная точка), затем выставим True там, где нет реальных данных
    src_key_padding_mask = torch.zeros((batch_size, max_len), dtype=torch.bool)

    y_class_list = []
    y_reg_list = []

    # 4) Копируем данные в паддинг-тензоры
    for i, (X, y_class, y_reg) in enumerate(batch):
        length = X.shape[0]
        padded_X[i, :length, :] = X
        # Для элементов после length делаем mask = True
        if length < max_len:
            src_key_padding_mask[i, length:] = True

        y_class_list.append(y_class)
        y_reg_list.append(y_reg)

    # 5) Склеиваем метки
    y_class_tensor = torch.stack(y_class_list, dim=0)  # [B, 8]
    y_reg_tensor   = torch.stack(y_reg_list, dim=0)   # [B, 7]

    return padded_X, src_key_padding_mask, y_class_tensor, y_reg_tensor

def train():
    # Создаём датасет
    dataset = SiamDataset(frame_test)
    # Создаём DataLoader
    loader = DataLoader(dataset, 
                        batch_size=8, 
                        shuffle=True, 
                        collate_fn=collate_fn_with_padding)


    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Device is {device}")
    # device = 'cpu'

    # model = ConvTransformer(d_model=96, nhead=16, num_layers=4, dim_feedforward=128, in_channels=1)
    model = ConvTransformer(d_model=60, nhead=10, num_layers=3, dim_feedforward=256, in_channels=3)
    model.to(device)

    optimizer = optim.Adam(model.parameters(), lr=3e-4)
    bce_loss = nn.BCEWithLogitsLoss()
    mse_loss = nn.L1Loss(reduction="mean")
    model.train()
    for epoch in range(25):
        total_loss = 0.0
        for batch_data in loader:
            # batch_data = (padded_X, src_key_padding_mask, y_class, y_reg)
            padded_X, src_mask, y_class, y_reg = batch_data
            # print(padded_X)
            padded_X = padded_X.to(device)
            src_mask = src_mask.to(device)
            # y_class  = y_class.to(device)
            # print(y_reg)
            y_reg = y_reg[:, 0].view(-1, 1)

            # print(y_reg)
            y_reg = y_reg.to(device)

            optimizer.zero_grad()

            # Прогон через модель
            # logits, reg_out = model(padded_X, src_key_padding_mask=src_mask)
            reg_out = model(padded_X, src_key_padding_mask=src_mask)
            # print(reg_out)
            # print(y_reg)

            # Лосс по классификации
            # loss_class = bce_loss(logits, y_class)
            # Лосс по регрессии
            loss_reg = mse_loss(reg_out, y_reg)
            # l1_norm = sum(p.abs().sum() for p in model.parameters())

            loss =  loss_reg  #+ l1_norm #+  loss_reg
            # print(loss)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
        print(f"Epoch {epoch+1}, loss={total_loss:.4f}")



In [ ]:
# model.train()
# for epoch in range(10):
#     total_loss = 0.0
#     for batch_data in loader:
#         # batch_data = (padded_X, src_key_padding_mask, y_class, y_reg)
#         padded_X, src_mask, y_class, y_reg = batch_data

#         padded_X = padded_X.to(device)
#         src_mask = src_mask.to(device)
#         # y_class  = y_class.to(device)
#         y_reg    = y_reg.to(device)
#         y_reg = y_reg[:, 0].view(-1, 1)
#         optimizer.zero_grad()

#         # Прогон через модель
#         # logits, reg_out = model(padded_X, src_key_padding_mask=src_mask)
#         # print(logits)
#         # print(y_class)
#         reg_out = model(padded_X, src_key_padding_mask=src_mask)
#         # Лосс по классификации
#         # loss_class = bce_loss(logits, y_class)
#         # Лосс по регрессии
#         loss_reg = mse_loss(reg_out, y_reg)
#         l1_norm = sum(p.abs().sum() for p in model.parameters())

#         loss = loss_reg   #+ l1_norm #+ loss_reg loss_class
#         # print(loss)
#         loss.backward()
#         optimizer.step()

#         total_loss += loss.item()
#     print(f"Epoch {epoch+1}, loss={total_loss:.4f}")

Epoch 1, loss=404.8748
Epoch 2, loss=412.4134
Epoch 3, loss=381.7196
Epoch 4, loss=396.7958
Epoch 5, loss=363.1393
Epoch 6, loss=363.7514
Epoch 7, loss=375.0607
Epoch 8, loss=416.9490
Epoch 9, loss=388.2826
Epoch 10, loss=341.3006


In [229]:
torch.cuda.empty_cache()
import gc
torch.cuda.reset_max_memory_allocated()  # Сброс максимального использования памяти
torch.cuda.reset_max_memory_cached()     # Сброс кэшированной памяти

gc.collect()

# Очистка кэша CUDA
torch.cuda.empty_cache()

d:\Python3\Lib\site-packages\torch\cuda\memory.py:391: FutureWarning: torch.cuda.reset_max_memory_allocated now calls torch.cuda.reset_peak_memory_stats, which resets /all/ peak memory stats.
  warnings.warn(
d:\Python3\Lib\site-packages\torch\cuda\memory.py:417: FutureWarning: torch.cuda.reset_max_memory_cached now calls torch.cuda.reset_peak_memory_stats, which resets /all/ peak memory stats.
  warnings.warn(


In [ ]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# # device = "cpu"
# # model.to(device=device)
# print(f"rinning at {device}")
# # dataset = SiamDataset(frame_test)
# loader = DataLoader(dataset, batch_size=1, shuffle=False, collate_fn=collate_fn_with_padding)
# all_proba = []
# all_real_y = []
# all_reg = []
# all_y_reg = []
# model.eval()
# with torch.no_grad():
#     for padded_X, src_mask, y_class, y_reg in loader:

#         padded_X = padded_X.to(device)
#         src_mask = src_mask.to(device)
#         # y_class  = y_class.to(device)
#         y_reg = y_reg.to(device)

#         optimizer.zero_grad()

#         # Прогон через модель
#         # logits, reg_out = model(padded_X, src_key_padding_mask=src_mask)
#         reg_out = model(padded_X, src_key_padding_mask=src_mask)
#         # print(logits)
#         # print(y_class)

#         # Лосс по классификации
#         # loss_class = bce_loss(logits, y_class)
#         all_reg.append(reg_out.cpu())
#         all_y_reg.append(y_reg.cpu())
        
#         pred_proba = torch.sigmoid(reg_out)
#         all_proba.append(pred_proba.cpu())
#         all_real_y.append(y_class.cpu())

rinning at cuda


In [ ]:

# all_real_y_2 = [tensor[0, 1].cpu().detach().numpy() for tensor in all_real_y]
# pred_proba_2 = [tensor.cpu().detach().numpy() for tensor in all_proba]
# all_answers_2 = [(pred_proba[0] > 0.5).int().cpu().detach().numpy() for pred_proba in all_proba]
# # # all_answers_2 = 
# print(len(all_real_y_2))
# print(len(all_answers_2))
# print(all_real_y_2)
# print(all_answers_2)

500
500
[array(1., dtype=float32), array(1., dtype=float32), array(0., dtype=float32), array(1., dtype=float32), array(1., dtype=float32), array(0., dtype=float32), array(0., dtype=float32), array(1., dtype=float32), array(0., dtype=float32), array(0., dtype=float32), array(1., dtype=float32), array(1., dtype=float32), array(1., dtype=float32), array(1., dtype=float32), array(1., dtype=float32), array(1., dtype=float32), array(1., dtype=float32), array(1., dtype=float32), array(1., dtype=float32), array(0., dtype=float32), array(1., dtype=float32), array(0., dtype=float32), array(1., dtype=float32), array(1., dtype=float32), array(0., dtype=float32), array(1., dtype=float32), array(1., dtype=float32), array(1., dtype=float32), array(1., dtype=float32), array(1., dtype=float32), array(0., dtype=float32), array(1., dtype=float32), array(1., dtype=float32), array(1., dtype=float32), array(1., dtype=float32), array(1., dtype=float32), array(1., dtype=float32), array(1., dtype=float32), arr

In [ ]:
# all_reg_2 = np.array([tmp_reg[0].cpu().detach().numpy() for tmp_reg in all_reg])[:, 0]
# all_y_reg_2 = np.array([tmp_reg[0].cpu().detach().numpy() for tmp_reg in all_y_reg])[:, 0]

# # np.mean(np.abs(np.array(all_y_reg_2) - np.array(all_reg_2)), axis=0)
# mask = all_y_reg_2 > 0
# all_reg_2[mask] = 0
# all_y_reg_2[mask] = 0
# # all_reg_2
# # all_y_reg_2
# np.mean(np.abs(np.array(all_y_reg_2) - np.array(all_reg_2)), axis=0)
# # np.abs(np.array(all_y_reg_2) - np.array(all_reg_2))

AttributeError: 'numpy.ndarray' object has no attribute 'cpu'

In [ ]:
# from sklearn.metrics import precision_score, recall_score, f1_score, hamming_loss
# precision = precision_score(all_real_y_2, all_answers_2, average='macro')

# recall = recall_score(all_real_y_2, all_answers_2, average='macro')

# f1 = f1_score(all_real_y_2, all_answers_2, average='macro')

# hamming = hamming_loss(all_real_y_2, all_answers_2)
# print(np.sum(np.all(np.array(all_answers_2) == np.array(all_real_y_2), axis=1))/len(all_answers_2))
# print(f"Precision: {precision}, Recall: {recall}, F1: {f1}, Hamming Loss: {hamming}")

0.0
Precision: 0.8338316095669037, Recall: 0.8237155001860884, F1: 0.8284938779771226, Hamming Loss: 0.138


In [3]:
# Оценка
def eval_model(reg_models:list[nn.Module], class_real_index:int):
    dataset = SiamDataset(frame_test)
    loader = DataLoader(dataset, batch_size=1, shuffle=False, collate_fn=collate_fn_with_padding)
    for model in reg_models:
        model.eval()

    all_mae = [[] for _ in range(n_class)]
    all_real_y = []
    all_reg = [[] for _ in range(n_class)]
    all_y_reg = []
    all_proba = [[] for _ in range(n_class)]

    with torch.no_grad():
        for batch_data in loader:
            padded_X, src_mask, y_class, y_reg = batch_data
            padded_X = padded_X.to(device)
            src_mask = src_mask.to(device)
            # y_class = y_class[:, 1:n_class + 1].to(device)
            y_reg = y_reg.to(device)

            # Предсказание классов
            # logits = classifier(padded_X, src_key_padding_mask=src_mask)
            # y_class_pred = (logits > 0).float()  # Бинарные предсказания

            # all_y_class_true.append(y_class.cpu())
            # all_y_class_pred.append(y_class_pred.cpu())

            # Оценка регрессии для каждого класса
            for i in range(n_class):
                # mask = y_class[:, i] == 1
                # if mask.any():
                    # padded_X_i = padded_X[mask]
                    # src_mask_i = src_mask[mask]
                    # y_reg_i = y_reg[mask, i]

                reg_out = reg_models[i](padded_X, src_key_padding_mask=src_mask).squeeze(-1)
                # mask = y_reg_i > 0
                # print(mask)
                mae = torch.abs(reg_out - y_reg[:, i + class_real_index].view(-1, 1)).mean().item()
                all_mae[i].append(mae)
                all_reg[i].append(reg_out.cpu().detach().numpy())
                
                
                pred_proba = torch.sigmoid(reg_out)
                all_proba.append(pred_proba.cpu().detach().numpy())
            all_y_reg.append(y_reg.cpu().detach().numpy())    
            all_real_y.append(y_class.cpu().detach().numpy())

    # Вычисление метрик
    # Средний MAE по классам
    mae_per_class = [np.mean(maes) if maes else 0 for maes in all_mae]

    # Точность классификации (доля правильно предсказанных меток)
    # all_y_class_true = torch.cat(all_y_class_true, dim=0)
    # all_y_class_pred = torch.cat(all_y_class_pred, dim=0)
    # accuracy = (all_y_class_true == all_y_class_pred).float().mean().item()

    # Вывод результатов
    # print(f"Точность предсказания классов: {accuracy:.4f}")
    for i in range(n_class):
        print(f"MAE для класса {i}: {mae_per_class[i]:.4f}")

    # print(len([i[0] for i in all_reg[0]]))
    for class_index in range(n_class):
    # class_index = 6
    # print(all_real_y)
        all_reg_2 = np.array([i[0] for i in all_reg[class_index]])
        all_y_reg_2 = np.array([i[0, class_index] for i in all_y_reg])
        # print(all_y_reg_2)
        # all_reg_2 = all_reg_2[:, 0]

        # np.mean(np.abs(np.array(all_y_reg_2) - np.array(all_reg_2)), axis=0)
        mask = all_y_reg_2 > 0
        all_reg_2[mask] = 0
        all_y_reg_2[mask] = 0
        # all_reg_2
        # all_y_reg_2
        print(f"class index {class_real_index} MAE:", np.mean(np.abs(np.array(all_y_reg_2) - np.array(all_reg_2)), axis=0))
        print(f"class index {class_real_index} Совпадения с MAE < 0.15:", np.sum((np.abs(np.array(all_y_reg_2) - np.array(all_reg_2))) <= 0.15) / len(dataset))
        # np.abs(np.array(all_y_reg_2) - np.array(all_reg_2))

    import os
    # Папка для сохранения моделей
    save_dir = "saved_models"
    os.makedirs(save_dir, exist_ok=True)

    # Сохраняем каждую модель
    for idx, model in enumerate(reg_models):
        # Сохраняем состояние модели
        model_path = os.path.join(save_dir, f"BESTTransmodelv2_{class_real_index}class_P_and_derivative{idx}.pt")
        torch.save(model.state_dict(), model_path)
        print(f"Модель для класса {idx} сохранена в {model_path}")

In [ ]:
# Устройство
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device is {device}")

# Создаем датасет и загрузчик (предполагается, что frame_test определен)

# Параметры
n_class = 1  # Используем обучение по 1 из 7 классов
n_epochs = 6

for class_index in range(5):
    dataset = SiamDataset(frame)
    loader = DataLoader(dataset, batch_size=4, shuffle=True, collate_fn=collate_fn_with_padding)

    class_index = 0

    

    # Инициализация моделей
    # classifier = ClassConvTransformer(in_channels=3, n_class=n_class).to(device)
    reg_models = [ConvTransformer(d_model=60, nhead=10, num_layers=2, dim_feedforward=128, in_channels=3).to(device) for _ in range(n_class)]

    # Оптимизаторы
    # optimizer_class = optim.Adam(classifier.parameters(), lr=3e-4)
    optimizers_reg = [optim.Adam(model.parameters(), lr=3e-4) for model in reg_models]

    # Функции потерь
    # bce_loss = nn.BCEWithLogitsLoss()
    mse_loss = nn.L1Loss(reduction="mean")

    # Обучение
    for epoch in range(n_epochs):
        for model in reg_models:
            model.train()
        
        total_loss_class = 0.0
        total_loss_reg = [0.0] * n_class
        
        for batch_data in loader:
            padded_X, src_mask, y_class, y_reg = batch_data
            padded_X = padded_X.to(device)
            src_mask = src_mask.to(device)
            # y_class = y_class[:, :n_class].to(device)  # Берем первые 7 меток
            # y_reg = y_reg.to(device)                   # [B, 7]

            # # Обучение классификатора
            # optimizer_class.zero_grad()
            # logits = classifier(padded_X, src_key_padding_mask=src_mask)
            # loss_class = bce_loss(logits, y_class)
            # loss_class.backward()
            # optimizer_class.step()
            # total_loss_class += loss_class.item()

            # Обучение регрессионных моделей
            for i in range(n_class):
                y_reg = y_reg[:,i + class_index].view(-1, 1).to(device)
                # mask = y_class[:, i+1] == 1  # Образцы, где класс i активен
                # print(i)
                # if mask.any():
                # padded_X_i = padded_X[mask]
                # # src_mask_i = src_mask[mask]
                # y_reg_i = y_reg[mask, i].view(-1, 1)

                optimizers_reg[i].zero_grad()
                reg_out = reg_models[i](padded_X, src_key_padding_mask=src_mask)
                loss_reg = mse_loss(reg_out, y_reg)
                loss_reg.backward()
                optimizers_reg[i].step()
                total_loss_reg[i] += loss_reg.item()
                del padded_X, src_mask, y_reg, reg_out
                torch.cuda.empty_cache()  # Освобождение кэша CUDA

        # Вывод логов
        print(f"Epoch {epoch+1}")
        for i in range(n_class):
            if total_loss_reg[i] > 0:
                print(f"    Class {i + class_index} Regression Loss = {total_loss_reg[i]:.4f}")


    eval_model(reg_models, class_real_index=class_index)
    del reg_models[0]
    torch.cuda.empty_cache()  # Освобождение кэша CUDA

Device is cuda


d:\Python3\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
d:\Python3\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
d:\Python3\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [ ]:
# print(len([i[0] for i in all_reg[0]]))
for class_index in range(n_class):
# class_index = 6
# print(all_real_y)
    all_reg_2 = np.array([i[0] for i in all_reg[class_index]])
    all_y_reg_2 = np.array([i[0, class_index] for i in all_y_reg])
    # print(all_y_reg_2)
    # all_reg_2 = all_reg_2[:, 0]

    # np.mean(np.abs(np.array(all_y_reg_2) - np.array(all_reg_2)), axis=0)
    mask = all_y_reg_2 > 0
    all_reg_2[mask] = 0
    all_y_reg_2[mask] = 0
    # all_reg_2
    # all_y_reg_2
    print(f"class index {class_index} MAE:", np.mean(np.abs(np.array(all_y_reg_2) - np.array(all_reg_2)), axis=0))
    print(np.sum((np.abs(np.array(all_y_reg_2) - np.array(all_reg_2))) <= 0.15) / len(dataset))
    # np.abs(np.array(all_y_reg_2) - np.array(all_reg_2))

class index 0 MAE: 0.7075205
0.726


In [ ]:
import os
# Папка для сохранения моделей
save_dir = "saved_models"
os.makedirs(save_dir, exist_ok=True)

# Сохраняем каждую модель
for idx, model in enumerate(reg_models):
    # Сохраняем состояние модели
    model_path = os.path.join(save_dir, f"BESTmodelv1_1classONLYderivative{idx}.pt")
    torch.save(model.state_dict(), model_path)
    print(f"Модель для класса {idx} сохранена в {model_path}")

Модель для класса 0 сохранена в saved_models\BESTmodelv1_0classONLYderivative0.pt


In [26]:
from sklearn.metrics import precision_score, recall_score, f1_score, hamming_loss
precision = precision_score(all_real_y_2, all_answers_2, average='macro')

recall = recall_score(all_real_y_2, all_answers_2, average='macro')

f1 = f1_score(all_real_y_2, all_answers_2, average='macro')

hamming = hamming_loss(all_real_y_2, all_answers_2)
print(np.sum(np.all(np.array(all_answers_2) == np.array(all_real_y_2), axis=1))/len(all_answers_2))
print(f"Precision: {precision}, Recall: {recall}, F1: {f1}, Hamming Loss: {hamming}")

NameError: name 'all_real_y_2' is not defined

In [27]:

all_real_y_2 = [tensor[0, 1].cpu().detach().numpy() for tensor in all_real_y]
pred_proba_2 = [tensor.cpu().detach().numpy() for tensor in all_proba]
all_answers_2 = [(pred_proba[0] > 0.5).int().cpu().detach().numpy() for pred_proba in all_proba]
# # all_answers_2 = 
print(len(all_real_y_2))
print(len(all_answers_2))
print(all_real_y_2)
print(all_answers_2)

AttributeError: 'numpy.float32' object has no attribute 'cpu'